In [ ]:
import matplotlib.pyplot as plt

def roc_plot(y_test, y_pred):
    from sklearn.metrics import roc_curve, auc
    from sklearn.calibration import label_binarize
    from itertools import cycle

    rice = {
        0: 'Cammeo',
        1: 'Osmanscik',
    }

    y_test = label_binarize(y_test, classes=[0, 1])
    n_classes = y_test.shape[1]

    fpr, tpr, roc_auc = dict(), dict(), dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    colors = cycle(['red', 'blue'])

    plt.figure(figsize=(10, 5))

    for i, color in zip(range(n_classes), colors):
        plt.subplot(1, 2, i + 1)
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                label=f'{i}. {rice[i]} (area = {roc_auc[i]:.2f})')

        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([-.05, 1.05])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'Poker Hands ROC - {rice[i]}')
        plt.legend(loc='lower right')

    plt.tight_layout()
    plt.show(block=True)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

cols = ['Area', 'Perimeter', 'Major_Axis_Length', 'Minor_Axis_Length',
        'Eccentricity', 'Convex_Area', 'Extent', 'Class']

data = pd.read_csv('./data/Rice_Cammeo_Osmancik.arff', names=cols)

In [ ]:
x = data.iloc[:, :7]
y = data['Class'].apply(lambda x: 0 if x == 'Cammeo' else 1).values

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=.25, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=50, verbose=1, max_depth=20,  random_state=42, oob_score=True)

rf_model.fit(x_train, y_train)

y_pred = rf_model.predict(x_test)
y_prob = rf_model.predict_proba(x_test)

In [ ]:
from sklearn.metrics import classification_report

roc_plot(y_test, y_prob)
print(classification_report(y_test, y_pred))